Limeng Yang & Rik van Rhee

Jupyter Notebook used for the measurements of our experiment.

In [138]:
# Import statements

import pandas as pd
import matplotlib.pyplot as plt
from lmfit import models
from scipy.signal import find_peaks

First we will use the calibration data to convert the time axis in our data to a frequency axis.

In [139]:
# Open calibration data

df_calib = pd.read_csv('Data 01-12-23/Calibration_Data.csv', header=None, usecols=[3,4,9,10])

#(Rename columns?)


In [140]:
# Show the data in a graph

plt.plot(df_calib[3],df_calib[4], label="Ch3")
plt.plot(df_calib[9],df_calib[10], label="Ch4")
plt.xlabel("Time (s)")
plt.ylabel("Intensity in PD (v)")
plt.legend()

To convert the time axis to frequency, we will fit these curves with a linear fit and two Gaussian fits. From this we can take the found center points of the Gaussian dips and compare the difference in these dips to the known difference in frequency of these. This is $6834.7$ MHz.

In [141]:
# Define the fitfunctions

def LinFunc(x, a, b):
    y = a*x+b
    return y

Lin_model = models.Model(LinFunc)
gauss_model_1 = models.GaussianModel(prefix="A_")
gauss_model_2 = models.GaussianModel(prefix="B_")

tot_model = Lin_model + gauss_model_1 + gauss_model_2

print(f'parameter names: {tot_model.param_names}')
print(f'independent variables: {tot_model.independent_vars}')

parameter names: ['a', 'b', 'A_amplitude', 'A_center', 'A_sigma', 'B_amplitude', 'B_center', 'B_sigma']
independent variables: ['x']


In [142]:
# Set some initial parameters from observation

a = (df_calib[4][0]-df_calib[4][len(df_calib[4])-1])/(df_calib[3][0]-df_calib[3][len(df_calib[3])-1]) 
b = 1.7
A_amp = -0.3
A_center = 0.20
A_sigma = 0.005
B_amp = -0.05
B_center = 0.27
B_sigma = 0.005

In [143]:
# Fit the data with the models
result_1= tot_model.fit(df_calib[4], x=df_calib[3], a=a, b=b, A_amplitude=A_amp, A_center=A_center, A_sigma=A_sigma, B_amplitude=-3.6906e-04, B_center=B_center, B_sigma=B_sigma)
result_2= tot_model.fit(df_calib[10], x=df_calib[9], a=a, b=b, A_amplitude=A_amp, A_center=A_center, A_sigma=A_sigma, B_amplitude=B_amp, B_center=B_center, B_sigma=B_sigma)


In [144]:
# Plot the first fit
plt.plot(df_calib[3], df_calib[4], label='data CH3')
plt.plot(df_calib[3], result_1.best_fit, '-', label='best fit CH3')
plt.legend()

In [145]:
# Plot the second fit
plt.plot(df_calib[9], df_calib[10], label='data CH4')
plt.plot(df_calib[9], result_2.best_fit, '-', label='best fit CH4')
plt.legend()

In [146]:
# Compare the centers of the two fits

A_center_1 = result_1.params.get("A_center").value
A_center_2 = result_2.params.get("A_center").value
B_center_1 = result_1.params.get("B_center").value
B_center_2 = result_2.params.get("B_center").value

print(f"The first peak in the two fits have centers at {A_center_1} and {A_center_2}")
print(f"The second peak in the two fits have centers at {B_center_1} and {B_center_2}")

The first peak in the two fits have centers at 0.202194582795992 and 0.202189933914028
The second peak in the two fits have centers at 0.26644218706949213 and 0.2670902247791182


These centers are close to each other, so we will take the mean

In [147]:
peak_A = (A_center_1+A_center_2)/2
peak_B = (B_center_1+B_center_2)/2

We can thus convert between the time axis and the frequency axis by using:
\begin{equation}
f = \frac{\Delta f}{\Delta t} t.
\end{equation}
We will use $\frac{\Delta f}{\Delta t}=Q$ for notation.


In [148]:
delta_f = 6834.7 #MHz
delta_t = abs(peak_A-peak_B)
Q = delta_f/delta_t
Q

105842.99484967359

In [149]:
# Converting the axis for the calibration data
df_calib["freq1"]=Q*df_calib[3]
df_calib["freq2"]=Q*df_calib[9]

# Plotting this data
plt.plot(df_calib["freq1"],df_calib[4], label="Ch3")
plt.plot(df_calib["freq2"],df_calib[10], label="Ch4")
plt.xlabel("Freq. (MHz)")
plt.ylabel("Intensity in PD (v)")
plt.legend()

For each dataset we will further use, we can thus convert the time axis to frequency axis.

We will now open each recorded measurement at different currents. We will find the peaks of the $F=3$ lines and calculate the difference. These differences we will add to a list which we will plot against a list of the used currents.

In [150]:
# First create the list of frequencies en list of currents
Currents = [0, 0.105, 0.206, 0.3, 0.408, 0.508, 0.6]
Frequencies = []

In [151]:
# First the dataset without a current
df_nocur =  pd.read_csv('Data 01-12-23/No_Current_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_nocur["freq1"]=df_nocur[3]*Q
df_nocur["freq2"]=df_nocur[9]*Q

In [152]:
# Plot this data

plt.plot(df_nocur["freq1"], df_nocur[4], label="CH3")
plt.plot(df_nocur["freq2"], df_nocur[10], label="CH4")

In [153]:
# Use an online tool to help find peaks
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_nocur[4], lookahead=100)
peaks2 = peakdetect(df_nocur[10], lookahead=100)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

In [154]:
# Plot the data with the found peaks
plt.plot(df_nocur["freq1"], df_nocur[4], "-")
plt.scatter(df_nocur["freq1"][indices1], df_nocur[4][indices1], color="red")

plt.plot(df_nocur["freq2"], df_nocur[10], "-")
plt.scatter(df_nocur["freq2"][indices2], df_nocur[10][indices2], color="red")

From this we can see that the peak we want is found. We will then, per datafile, manually get this peak and add the difference to a list.

In [155]:
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_nocur["freq1"][index]<21500:
        sive1_freq.append(df_nocur["freq1"][index])
        sive1_inten.append(df_nocur[4][index])
for index in indices2:
    if 21300<df_nocur["freq2"][index]<21500:
        sive2_freq.append(df_nocur["freq1"][index])
        sive2_inten.append(df_nocur[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [156]:
# Now data with about 1 Ampere
df_1Amp =  pd.read_csv('Data 01-12-23/0.105Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_1Amp["freq1"]=df_1Amp[3]*Q
df_1Amp["freq2"]=df_1Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_1Amp[4], lookahead=100)
peaks2 = peakdetect(df_1Amp[10], lookahead=100)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_1Amp["freq1"], df_1Amp[4], "-")
plt.scatter(df_1Amp["freq1"][indices1], df_1Amp[4][indices1], color="red")
plt.plot(df_1Amp["freq2"], df_1Amp[10], "-")
plt.scatter(df_1Amp["freq2"][indices2], df_1Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_1Amp["freq1"][index]<21500:
        sive1_freq.append(df_1Amp["freq1"][index])
        sive1_inten.append(df_1Amp[4][index])
for index in indices2:
    if 21300<df_1Amp["freq2"][index]<21500:
        sive2_freq.append(df_1Amp["freq1"][index])
        sive2_inten.append(df_1Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [157]:
# Now data with about 2 Ampere
df_2Amp =  pd.read_csv('Data 01-12-23/0.206Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_2Amp["freq1"]=df_2Amp[3]*Q
df_2Amp["freq2"]=df_2Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_2Amp[4], lookahead=100)
peaks2 = peakdetect(df_2Amp[10], lookahead=100)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_2Amp["freq1"], df_2Amp[4], "-")
plt.scatter(df_2Amp["freq1"][indices1], df_2Amp[4][indices1], color="red")
plt.plot(df_2Amp["freq2"], df_2Amp[10], "-")
plt.scatter(df_2Amp["freq2"][indices2], df_2Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_2Amp["freq1"][index]<21500:
        sive1_freq.append(df_2Amp["freq1"][index])
        sive1_inten.append(df_2Amp[4][index])
for index in indices2:
    if 21300<df_2Amp["freq2"][index]<21500:
        sive2_freq.append(df_2Amp["freq1"][index])
        sive2_inten.append(df_2Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [158]:
# Now data with about 3 Ampere
df_3Amp =  pd.read_csv('Data 01-12-23/0.300Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_3Amp["freq1"]=df_3Amp[3]*Q
df_3Amp["freq2"]=df_3Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_3Amp[4], lookahead=100)
peaks2 = peakdetect(df_3Amp[10], lookahead=100)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_3Amp["freq1"], df_3Amp[4], "-")
plt.scatter(df_3Amp["freq1"][indices1], df_3Amp[4][indices1], color="red")
plt.plot(df_3Amp["freq2"], df_3Amp[10], "-")
plt.scatter(df_3Amp["freq2"][indices2], df_3Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_3Amp["freq1"][index]<21500:
        sive1_freq.append(df_3Amp["freq1"][index])
        sive1_inten.append(df_3Amp[4][index])
for index in indices2:
    if 21300<df_3Amp["freq2"][index]<21500:
        sive2_freq.append(df_3Amp["freq1"][index])
        sive2_inten.append(df_3Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [159]:
# Now data with about 4 Ampere
df_4Amp =  pd.read_csv('Data 01-12-23/0.408Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_4Amp["freq1"]=df_4Amp[3]*Q
df_4Amp["freq2"]=df_4Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_4Amp[4], lookahead=100)
peaks2 = peakdetect(df_4Amp[10], lookahead=100)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_4Amp["freq1"], df_4Amp[4], "-")
plt.scatter(df_4Amp["freq1"][indices1], df_4Amp[4][indices1], color="red")
plt.plot(df_4Amp["freq2"], df_4Amp[10], "-")
plt.scatter(df_4Amp["freq2"][indices2], df_4Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_4Amp["freq1"][index]<21500:
        sive1_freq.append(df_4Amp["freq1"][index])
        sive1_inten.append(df_4Amp[4][index])
for index in indices2:
    if 21300<df_4Amp["freq2"][index]<21500:
        sive2_freq.append(df_4Amp["freq1"][index])
        sive2_inten.append(df_4Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [160]:
# Now data with about 5 Ampere
df_5Amp =  pd.read_csv('Data 01-12-23/0.509Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_5Amp["freq1"]=df_5Amp[3]*Q
df_5Amp["freq2"]=df_5Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_5Amp[4], lookahead=100)
peaks2 = peakdetect(df_5Amp[10], lookahead=50)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_5Amp["freq1"], df_5Amp[4], "-")
plt.scatter(df_5Amp["freq1"][indices1], df_5Amp[4][indices1], color="red")
plt.plot(df_5Amp["freq2"], df_5Amp[10], "-")
plt.scatter(df_5Amp["freq2"][indices2], df_5Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_5Amp["freq1"][index]<21500:
        sive1_freq.append(df_5Amp["freq1"][index])
        sive1_inten.append(df_5Amp[4][index])
for index in indices2:
    if 21300<df_5Amp["freq2"][index]<21500:
        sive2_freq.append(df_5Amp["freq1"][index])
        sive2_inten.append(df_5Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

In [161]:
# Now data with about 6 Ampere
df_6Amp =  pd.read_csv('Data 01-12-23/0.600Amp_Data.csv', header=None, usecols=[3,4,9,10])

# Convert the time to freq
df_6Amp["freq1"]=df_6Amp[3]*Q
df_6Amp["freq2"]=df_6Amp[9]*Q

# Use tool
from peakdetect_online import peakdetect
peaks1 = peakdetect(df_6Amp[4], lookahead=100)
peaks2 = peakdetect(df_6Amp[10], lookahead=50)

indices1 = []
indices2 = []
# Filter out only the indices
for p in peaks1:
    for d in p:
        indices1.append(d[0])
for p in peaks2:
    for d in p:
        indices2.append(d[0])

# Plot the data with the found peaks
plt.plot(df_6Amp["freq1"], df_6Amp[4], "-")
plt.scatter(df_6Amp["freq1"][indices1], df_6Amp[4][indices1], color="red")
plt.plot(df_6Amp["freq2"], df_6Amp[10], "-")
plt.scatter(df_6Amp["freq2"][indices2], df_6Amp[10][indices2], color="red")

# Sive out the desired peaks and find the differnce in frequency
sive1_freq=[]
sive1_inten=[]
sive2_freq=[]
sive2_inten=[]

for index in indices1:
    if 21300<df_6Amp["freq1"][index]<21500:
        sive1_freq.append(df_6Amp["freq1"][index])
        sive1_inten.append(df_6Amp[4][index])
for index in indices2:
    if 21300<df_6Amp["freq2"][index]<21500:
        sive2_freq.append(df_6Amp["freq1"][index])
        sive2_inten.append(df_6Amp[4][index])
        
for index in range(len(sive1_freq)-1):
    if sive1_inten[index]==max(sive1_inten):
        save1_index=index
for index in range(len(sive2_freq)-1):
    if sive2_inten[index]==max(sive2_inten):
        save2_index=index

diff = abs(sive1_freq[save1_index]-sive2_freq[save2_index])
Frequencies.append(diff)

We now have 2 lists, one with currents and one with difference of frequencies. We can now plot this.

In [162]:
# Convert from MHz to Hz
Frequencies_in_Hz=[]
for freq in Frequencies:
    Frequencies_in_Hz.append(freq*1e6)


plt.scatter(Currents, Frequencies_in_Hz)

This looks quite linear. And according to our hypothesis, the relation should be:
\begin{equation}
\Delta\nu = \frac{\mu_B\mu_0N\Delta M}{lh}I.
\end{equation}
Where that fractioned term is a constant.

We will now try to fit a linear fit through this data.

In [163]:
# Use the previous linear fit
LinFit = Lin_model.fit(Frequencies_in_Hz, x=Currents, a=6e6, b=0)

print(LinFit.fit_report())

plt.plot(Currents, Frequencies_in_Hz, label="data")
plt.plot(Currents, LinFit.best_fit, '-', label='best fit')
plt.legend()


[[Model]]
    Model(LinFunc)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 7
    # variables        = 2
    chi-square         = 2.6761e+13
    reduced chi-square = 5.3522e+12
    Akaike info crit   = 206.804416
    Bayesian info crit = 206.696236
    R-squared          = 0.97434958
[[Variables]]
    a:  60073121.7 +/- 4358981.69 (7.26%) (init = 6000000)
    b:  2007978.69 +/- 1587111.53 (79.04%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b) = -0.835


In [164]:
# Hypothetical constant
MuB = 9.274e-24
Mu0=1.2566e-6
N=100
DeltaM= #Yet not known, might need to find
l=0.01 #Yet not known
h=6.626e-34

const=(MuB*Mu0*N*DeltaM)/(l*h)
const

SyntaxError: invalid syntax (2217110932.py, line 5)